In [ ]:
# Importing nessesary libraries
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [9]:
# Obtaining predictions and evaluating performance

best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'\nAccuracy on test set: {accuracy:.2f}')
print('Classification Report:')
print(classification_report(y_test, y_pred))
print('Confusion Matrix:')
print(confusion_matrix(y_test, y_pred))

from sklearn.model_selection import cross_val_score
cv_scores = cross_val_score(best_model, X_train, y_train, cv=5)
print(f'\nCross-validation scores (10 folds): {cv_scores}')
print(f'Mean cross-validation score: {cv_scores.mean():.2f}')

# Data Exploration
df = pd.read_csv('cleaned_titanic.csv')
print(df.head())
print(df.describe())
print(df.info())


   PassengerId  Survived  Pclass  \
0          892         0       3   
1          893         1       3   
2          894         0       2   
3          895         0       3   
4          896         1       3   

                                           Name     Sex   Age  SibSp  Parch  \
0                              Kelly, Mr. James    male  34.5      0      0   
1              Wilkes, Mrs. James (Ellen Needs)  female  47.0      1      0   
2                     Myles, Mr. Thomas Francis    male  62.0      0      0   
3                              Wirz, Mr. Albert    male  27.0      0      0   
4  Hirvonen, Mrs. Alexander (Helga E Lindqvist)  female  22.0      1      1   

    Ticket     Fare Embarked  family  sex  
0   330911   7.8292        Q       0  NaN  
1   363272   7.0000        S       1  NaN  
2   240276   9.6875        Q       0  NaN  
3   315154   8.6625        S       0  NaN  
4  3101298  12.2875        S       2  NaN  
       PassengerId    Survived      Pclass  

In [10]:
# Missing Value Imputation
df.drop(columns=['sex'], inplace=True)
df['Sex'] = df['Sex'].map({'male': 1, 'female': 0})
df['Age'] = df['Age'].fillna(df['Age'].median())


In [11]:
# Target and Features selection
features = ['Pclass', 'Age', 'SibSp', 'Parch']
X = df[features]
y = df['Survived']


In [12]:
# Splitting data into Test and Train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [13]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [15]:
# Hyperparameter Tuning , Selction using Grid search CV
param_grid = {
    'n_estimators': [200, 300, 400, 500],
    'max_depth': [None, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False],
}

model = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_score = grid_search.best_score_
print("Best parameters found:", best_params)
print("Best cross-validation score:", best_score)


Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Best parameters found: {'bootstrap': True, 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}
Best cross-validation score: 0.6947535052012664


In [16]:
# Performance evaluation

best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'\nAccuracy on test set: {accuracy:.2f}')
print('Classification Report:')
print(classification_report(y_test, y_pred))
print('Confusion Matrix:')
print(confusion_matrix(y_test, y_pred))

# Cross-validation scores
from sklearn.model_selection import cross_val_score
cv_scores = cross_val_score(best_model, X_train, y_train, cv=5)
print(f'\nCross-validation scores (10 folds): {cv_scores}')
print(f'Mean cross-validation score: {cv_scores.mean():.2f}')



Accuracy on test set: 0.64
Classification Report:
              precision    recall  f1-score   support

           0       0.71      0.74      0.72        53
           1       0.52      0.48      0.50        31

    accuracy                           0.64        84
   macro avg       0.61      0.61      0.61        84
weighted avg       0.64      0.64      0.64        84

Confusion Matrix:
[[39 14]
 [16 15]]

Cross-validation scores (10 folds): [0.62686567 0.74626866 0.6119403  0.74626866 0.74242424]
Mean cross-validation score: 0.69
